In [8]:
print(f"\U0001F4BB Finished.")

💻 Finished.


In [2]:
def arg_test(*args):
    for l in args:
        print(l)

arg_test(
    # Layer parameters: name, nodes, activation
    ("input", 1, 4),
    ("hidden", 4, 8),
    ("hidden", 8, 8),
    ("output", 8, 4)
)


('input', 1, 4)
('hidden', 4, 8)
('hidden', 8, 8)
('output', 8, 4)


In [20]:
import pandas as pd
import csv
df = pd.read_csv("data/iris_str.csv", header=0, delimiter=',') 
classes = df.iloc[:,-1].unique().tolist()
df.iloc[:,-1] = df.iloc[:,-1].apply(lambda c: classes.index(c))
new_data = df.to_numpy()

new_data

array([[4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.6, 0.2, 0. ],
       [4.8, 3

In [2]:
import itertools

l1 = ['a', 'b', 'c']
l2 = [1]
l3 = ['zz', 'yy']

list(itertools.product(*[l1, l2, l3]))


[('a', 1, 'zz'),
 ('a', 1, 'yy'),
 ('b', 1, 'zz'),
 ('b', 1, 'yy'),
 ('c', 1, 'zz'),
 ('c', 1, 'yy')]

In [ ]:
import numpy as np

target = np.array(['dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'dog', 
    'cat', 'cat', 'hamster', 'hamster'])

def one_hot(array):
    unique, inverse = np.unique(array, return_inverse=True)
    print(unique, type(unique))
    onehot = np.eye(unique.shape[0])[inverse]
    return onehot

print(one_hot(target))
print(type(one_hot(target)))

In [ ]:
class Dataset:
    """
    Contains a set of data and the test, train, validation sets.
    """
    data: 'np.ndarray[float]' = []
    target_idx: int = -1  # Default to last

    def __repr__(self) -> str:
        """Pretty string of an array"""
        return np.array2string(self.data, formatter={'float_kind':lambda x: "%.4f" % x})


In [2]:
config = {
    # Setup
    "data_file": 'data/my_data.csv',  # Relative filepath to csv file
    "data_seperator": ' ',  # Usually ' ' or ','
    "header_rows": 0,  # Number of rows to ignore at start of file for header
    "random_seed": 2021,  # Used to initiate the numpy random generator

    # Dataset
    "train_test_split": (.33, .33), # val is remaining, if test = 0 will use '66/33' method
    "target": -1,  # Target index, -1 being last column
    "normalize_values": (-1, 1),  # Min and Max values
    "normalize_range": (0, -1),  # Index range to normalize, 'None' to skip normalisation

    # Model settings
    "input_nodes": 12,  # Number of input values (features)
    "output_nodes": 3,  # Number of output values (classes)
    "hidden_layers": 2,  # Number of hidden layers
    "activator": 'ann.Softmax',  # alternatives: ann.Relu, ann.Sigmoid

    # Parameters - multiple values in the list create a grid search
    "eta": [0.1],  # Training rate
    "hidden_nodes": [8],  # Hidden nodes
    "max_epochs": [150],  # Number of epochs to train
}

print(config)

{'data_file': 'data/my_data.csv', 'data_seperator': ' ', 'header_rows': 0, 'random_seed': 2021, 'train_test_split': (0.33, 0.33), 'target': -1, 'normalize_values': (-1, 1), 'normalize_range': (0, -1), 'input_nodes': 12, 'output_nodes': 3, 'hidden_layers': 2, 'activator': 'ann.Softmax', 'eta': [0.1], 'hidden_nodes': [8], 'max_epochs': [150]}
